# BuzzBlog Experiment Analysis

## Notebook Configuration

In [ ]:
########## GENERAL
# Environment (options: "colab", "local")
ENV = "local"
# Experiment directory name
EXPERIMENT = "BuzzBlogBenchmark_2021-03-18-14-14-16"

########## DIRECTORY STRUCTURE
LOADGEN_NODE = "apt087.apt.emulab.net"
LOADBALANCER_NODE = "apt078.apt.emulab.net"
APIGATEWAY_NODE = "apt078.apt.emulab.net"
ACCOUNT_SERVICE_NODE = "apt083.apt.emulab.net"
ACCOUNT_DB_NODE = "apt080.apt.emulab.net"
FOLLOW_SERVICE_NODE = "apt083.apt.emulab.net"
LIKE_SERVICE_NODE = "apt083.apt.emulab.net"
POST_SERVICE_NODE = "apt083.apt.emulab.net"
POST_DB_NODE = "apt080.apt.emulab.net"
UNIQUEPAIR_SERVICE_NODE = "apt083.apt.emulab.net"
UNIQUEPAIR_DB_NODE = "apt080.apt.emulab.net"

########## REQUEST LOGS
# Fine-grained window to group PIT data
PIT_FG_WINDOW_IN_MS = 50
# Function to aggregate PIT data
PIT_AGGREGATE_FUNC = "max"

########## SYSTEM EVENT MONITORING LOGS
##### TCPLIFE
# Fine-grained window to group TCP connection lifespan
TCPLIFE_FG_WINDOW_IN_MS = 25

########## SYSTEM RESOURCE MONITORING LOGS
##### COLLECTL
# Fine-grained window to group Collectl measurements
COLLECTL_FG_WINDOW_IN_MS = 50
# Function to aggregate Collectl measurements
COLLECTL_AGGREGATE_FUNC = "max"
# CPU metric to be analyzed (options: "user", "nice", "system", "wait", "irq", "soft", "steal", "idle", "total", "guest", "guest_n", "intrpt")
CPU_METRIC = "total"

## Notebook Setup

In [ ]:
# Import libraries
%matplotlib inline
import codecs
import csv
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sys
import tarfile
import time
import yaml
import warnings
warnings.filterwarnings("ignore")

# Set the environment
if ENV == "colab":
  from google.colab import drive
  drive.mount("GoogleDrive", force_remount=True)
  EXPERIMENT_DIR = os.path.join("GoogleDrive", "MyDrive", EXPERIMENT)
elif ENV == "local":
  EXPERIMENT_DIR = EXPERIMENT

# Load experiment metadata
with open(os.path.join(EXPERIMENT_DIR, "metadata.yml")) as metadata_file:
    metadata = yaml.load(metadata_file, Loader=yaml.Loader)

# Load experiment configuration files
with open(os.path.join(EXPERIMENT_DIR, "conf", "system.yml")) as system_conf_file:
    system_conf = yaml.load(system_conf_file, Loader=yaml.Loader)
with open(os.path.join(EXPERIMENT_DIR, "conf", "workload.yml")) as workload_conf_file:
    workload_conf = yaml.load(workload_conf_file, Loader=yaml.Loader)

## Experiment Metadata

In [ ]:
print(yaml.dump(metadata, default_flow_style=False))

## Request Logs

In [ ]:
# Parse request logs
REQUEST_LOG_PATTERN = r"^\[(\d+\-\d+\-\d+ \d+:\d+:\d+.\d+)\] (.+) (.+) (\d+) - latency=(\d+.\d+)$"
URL_PATTERN = r"^http://[\w\.]+:\d+/{path}/?\??{qs}$"
REQUEST_TO_TYPE = {
    (URL_PATTERN.format(path="account", qs=""), "POST"): "create_account",
    (URL_PATTERN.format(path="account/\d+", qs=""), "PUT"): "update_account",
    (URL_PATTERN.format(path="post", qs=""), "POST"): "create_post",
    (URL_PATTERN.format(path="post/\d+", qs=""), "DELETE"): "delete_post",
    (URL_PATTERN.format(path="follow", qs=""), "POST"): "follow_account",
    (URL_PATTERN.format(path="follow/\d+", qs=""), "DELETE"): "delete_follow",
    (URL_PATTERN.format(path="like", qs=""), "POST"): "like_post",
    (URL_PATTERN.format(path="like/\d+", qs=""), "DELETE"): "delete_like",
    (URL_PATTERN.format(path="post", qs=""), "GET"): "retrieve_recent_posts",
    (URL_PATTERN.format(path="post/\d+", qs=""), "GET"): "retrieve_post",
    (URL_PATTERN.format(path="like", qs="post_id=\d+"), "GET"): "retrieve_post_likes",
    (URL_PATTERN.format(path="account/\d+", qs=""), "GET"): "retrieve_account",
    (URL_PATTERN.format(path="post", qs="author_id=\d+"), "GET"): "retrieve_account_posts",
    (URL_PATTERN.format(path="follow", qs="followee_id=\d+"), "GET"): "retrieve_account_followers",
    (URL_PATTERN.format(path="follow", qs="follower_id=\d+"), "GET"): "retrieve_account_followees",
    (URL_PATTERN.format(path="like", qs="account_id=\d+"), "GET"): "retrieve_account_likes"
}
requests = []
tarball = tarfile.open(os.path.join(EXPERIMENT_DIR, "logs", LOADGEN_NODE, "loadgen.tar.gz"))
with tarball.extractfile("./loadgen.log") as loadgen_log_file:
  for request_log in loadgen_log_file:
    timestamp, method, url, status_code, latency = re.match(REQUEST_LOG_PATTERN, request_log.decode("utf-8")).groups()
    requests.append({"timestamp": pd.to_datetime(timestamp), "method": method, "url": url,
                     "status_code": int(status_code), "latency": float(latency)})

In [ ]:
# Build data frame
requests = pd.DataFrame(requests)
requests.sort_values(by="timestamp", ascending=True, inplace=True)
requests["status"] = requests.apply(lambda r: "successful" if r["status_code"] == 200 else "failed", axis=1)
requests["time"] = requests.apply(lambda r: (r["timestamp"] - requests["timestamp"].min()).total_seconds(), axis=1)
requests["window"] = requests.apply(lambda r: int(r["time"]), axis=1)
requests["fg_window"] = requests.apply(lambda r: int(r["time"] * 1000) // PIT_FG_WINDOW_IN_MS, axis=1)
requests["type"] = requests.apply(lambda r: [request_type
    for ((pattern, method), request_type) in REQUEST_TO_TYPE.items()
    if method == r["method"] and re.match(pattern, r["url"])][0], axis=1)
requests["rw"] = requests.apply(lambda r: "read" if r["method"] == "GET" else "write", axis=1)

### Workload Analysis

#### Summary

In [ ]:
print("Number of requests")
print("  Total:       %7d" % requests.shape[0])
print("  Status")
print("    Failed:    %7d (%9.5f%%)" % (requests[requests["status"] == "failed"]["status"].count(), (requests[requests["status"] == "failed"]["status"].count() / requests.shape[0]) * 100))
print("    Succesful: %7d (%9.5f%%)" % (requests[requests["status"] == "successful"]["status"].count(), (requests[requests["status"] == "successful"]["status"].count() / requests.shape[0]) * 100))
print("  Type")
print("    Read:      %7d (%9.5f%%)" % (requests[requests["rw"] == "read"]["rw"].count(), (requests[requests["rw"] == "read"]["rw"].count() / requests.shape[0]) * 100))
print("    Write:     %7d (%9.5f%%)" % (requests[requests["rw"] == "write"]["rw"].count(), (requests[requests["rw"] == "write"]["rw"].count() / requests.shape[0]) * 100))
print("Experiment duration (s)")
print("  Total:       %7.3f" % requests["time"].max())
print("  Ramp")
print("    Up:        %7.3f (%9.5f%%)" % (workload_conf["duration"]["ramp_up"], (workload_conf["duration"]["ramp_up"] / requests["time"].max()) * 100))
print("    Down:      %7.3f (%9.5f%%)" % (workload_conf["duration"]["ramp_down"], (workload_conf["duration"]["ramp_down"] / requests["time"].max()) * 100))
print("Latency (ms)")
print("  P99:         %7.2f" % (requests[requests["status"] == "successful"]["latency"].quantile(0.99) * 1000))
print("  P95:         %7.2f" % (requests[requests["status"] == "successful"]["latency"].quantile(0.95) * 1000))
print("  P50:         %7.2f" % (requests[requests["status"] == "successful"]["latency"].quantile(0.50) * 1000))
print("  Avg:         %7.2f" % (requests[requests["status"] == "successful"]["latency"].mean() * 1000))
print("  Std:         %7.2f" % (requests[requests["status"] == "successful"]["latency"].std() * 1000))
print("Throughput (req/s)")
print("  P99:         %7.2f" % requests.groupby(["window"])["window"].count().reindex(range(0, int(requests["time"].max()) + 1), fill_value=0).quantile(0.99))
print("  P95:         %7.2f" % requests.groupby(["window"])["window"].count().reindex(range(0, int(requests["time"].max()) + 1), fill_value=0).quantile(0.95))
print("  P50:         %7.2f" % requests.groupby(["window"])["window"].count().reindex(range(0, int(requests["time"].max()) + 1), fill_value=0).quantile(0.59))
print("  Avg:         %7.2f" % requests.groupby(["window"])["window"].count().reindex(range(0, int(requests["time"].max()) + 1), fill_value=0).mean())
print("  Std:         %7.2f" % requests.groupby(["window"])["window"].count().reindex(range(0, int(requests["time"].max()) + 1), fill_value=0).std())

In [ ]:
fig = plt.figure(figsize=(18, 6))
# Number of read/write requests
df = requests.groupby(["rw"]).count()["method"]
ax = fig.add_subplot(1, 3, 1)
df.plot(ax=ax, kind="pie", title="Number of read/write requests",
        xlabel="", ylabel="",
        legend=True)
# Number of successful/failed requests
df = requests.groupby(["status"]).count()["method"]
ax = fig.add_subplot(1, 3, 2)
df.plot(ax=ax, kind="pie", title="Number of successful/failed requests",
        xlabel="", ylabel="",
        legend=True)
# HTTP status code of failed requests
df = requests[requests["status"] == "failed"].groupby(["status_code"]).count()["method"]
ax = fig.add_subplot(1, 3, 3)
df.plot(ax=ax, kind="pie", title="HTTP status code of failed requests",
        xlabel="", ylabel="",
        legend=True)

In [ ]:
df = requests.groupby(["type", "status"]).count()["method"].unstack().fillna(0)
fig = plt.figure(figsize=(18, 12))
ax = fig.gca()
ax.grid(alpha=0.75)
df.plot(ax=ax, kind="bar", stacked=True, title="Number of requests of each type",
        xlabel="", ylabel="Requests (count)",
        color={"failed": "red", "successful": "blue"}, legend=True,
        grid=True)

### Throughput (1-second window)

In [ ]:
df = requests.groupby(["window", "status"])["window"].count().unstack().fillna(0)
df = df.reindex(range(0, int(df.index.max()) + 1), fill_value=0)
fig = plt.figure(figsize=(18, 12))
ax = fig.gca()
ax.grid(alpha=0.75)
ax.axvline(x=workload_conf["duration"]["ramp_up"],
           ls="--", color="green")
ax.axvline(x=workload_conf["duration"]["total"] - workload_conf["duration"]["ramp_down"],
           ls="--", color="green")
df.plot(ax=ax, kind="bar", stacked=True, title="Throughput (requests per second)",
        xlabel="Time (seconds)", ylabel="Requests (count)",
        color={"failed": "red", "successful": "blue"}, legend=True, grid=True,
        xticks=range(0, int(requests["time"].max()) + 1, 60))

In [ ]:
fig = plt.figure(figsize=(24, 24))
for (i, (request_type, _)) in enumerate(requests.groupby(["type"])):
    df = requests[requests["type"] == request_type]
    df = df.groupby(["window", "status"])["window"].count().unstack().fillna(0)
    df = df.reindex(range(0, int(requests["time"].max()) + 1), fill_value=0)
    ax = fig.add_subplot(4, 4, i + 1)
    ax.axvline(x=workload_conf["duration"]["ramp_up"],
               ls="--", color="green")
    ax.axvline(x=workload_conf["duration"]["total"] - workload_conf["duration"]["ramp_down"],
               ls="--", color="green")
    ax.grid(alpha=0.75)
    df.plot(ax=ax, kind="bar", stacked=True, title=request_type,
            xlabel="", ylabel="",
            color={"failed": "red", "successful": "blue"}, legend=False, grid=True,
            xticks=range(0, int(requests["time"].max()) + 1, 60))

### Throughput (fine-grained window)

In [ ]:
# [CONFIG] Request type or '*' for all
REQUEST_TYPE = '*'

df = requests[requests["type"] == REQUEST_TYPE] if REQUEST_TYPE != '*' else requests
df = df.groupby(["fg_window", "status"])["fg_window"].count().unstack().fillna(0)
df = df.reindex(range(0, int(df.index.max()) + 1), fill_value=0)
fig = plt.figure(figsize=(108, 48))
ax = fig.gca()
ax.grid(alpha=0.75)
ax.axvline(x=(workload_conf["duration"]["ramp_up"] * 1000) // PIT_FG_WINDOW_IN_MS,
           ls="--", linewidth=5, color="green")
ax.axvline(x=((workload_conf["duration"]["total"] - workload_conf["duration"]["ramp_down"])  * 1000) // PIT_FG_WINDOW_IN_MS,
           ls="--", linewidth=5, color="green")
ax.set_xticklabels([str(x * 60) for x in range(0, int(df.index.max()) // (60 * (1000 // PIT_FG_WINDOW_IN_MS)) + 1)])
df.plot(ax=ax, kind="bar", stacked=True,
        title="Throughput (requests per %s milliseconds)" % PIT_FG_WINDOW_IN_MS,
        xlabel="Time (seconds)", ylabel="Requests (count)",
        color={"failed": "red", "successful": "blue"}, legend=True, grid=True,
        xticks=range(0, int(df.index.max()) + 1, 60 * (1000 // PIT_FG_WINDOW_IN_MS)))

### Latency Distribution of Successful Requests Excluding Ramping Periods

In [ ]:
# [CONFIG] Max latency
MAX_LATENCY_IN_S = 10
# [CONFIG] Latency bin size
LATENCY_BIN_IN_MS = 50

df = requests[requests["status"] == "successful"]
df = df[(df["time"] >= workload_conf["duration"]["ramp_up"]) &
        (df["time"] <= workload_conf["duration"]["total"] - workload_conf["duration"]["ramp_down"])]
df["latency_bin"] = df.apply(lambda r: int(r["latency"] * 1000 // LATENCY_BIN_IN_MS), axis=1)
fig = plt.figure(figsize=(18, 12))
ax = fig.gca(xlabel="Latency (s)", ylabel="Requests (count)")
ax.grid(alpha=0.75)
ax.set_yscale("log")
ax.set_xlim((0, (1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S))
ax.set_ylim((0, 10000))
ax.set_xticks(range(0, (1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S + 1, (1000 // LATENCY_BIN_IN_MS)))
ax.set_xticklabels([str(s) for s in range(MAX_LATENCY_IN_S + 1)])
df["latency_bin"].plot(ax=ax, kind="hist",
                       title="Latency Distribution of Successful Requests Excluding Ramping Periods",
                       bins=range((1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S), grid=True)

In [ ]:
fig = plt.figure(figsize=(24, 24))
for (i, (request_type, _)) in enumerate(requests.groupby(["type"])):
    df = requests[(requests["status"] == "successful") & (requests["type"] == request_type)]
    df = df[(df["time"] >= workload_conf["duration"]["ramp_up"]) &
            (df["time"] <= workload_conf["duration"]["total"] - workload_conf["duration"]["ramp_down"])]
    df["latency_bin"] = df.apply(lambda r: int(r["latency"] * 1000 // LATENCY_BIN_IN_MS), axis=1)
    ax = fig.add_subplot(4, 4, i + 1)
    ax.grid(alpha=0.75)
    ax.set_yscale("log")
    ax.set_xlim((0, (1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S))
    ax.set_ylim((0, 10000))
    ax.set_xticks(range(0, (1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S + 1, (1000 // LATENCY_BIN_IN_MS)))
    ax.set_xticklabels([str(s) for s in range(MAX_LATENCY_IN_S + 1)])
    df["latency_bin"].plot(ax=ax, kind="hist", title=request_type,
                           xlabel="", ylabel="",
                           bins=range((1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S), grid=True)

### PIT Latency of Successful Requests (1-second window)

In [ ]:
df = requests[requests["status"] == "successful"]
df = df.groupby(["window"])["latency"].agg(PIT_AGGREGATE_FUNC)
df = df.reindex(range(0, int(df.index.max()) + 1), fill_value=0)
fig = plt.figure(figsize=(18, 12))
ax = fig.gca()
ax.axvline(x=workload_conf["duration"]["ramp_up"],
           ls="--", color="green")
ax.axvline(x=workload_conf["duration"]["total"] - workload_conf["duration"]["ramp_down"],
           ls="--", color="green")
ax.grid(alpha=0.75)
df.plot(ax=ax, kind="bar", title="PIT Latency of Successful Requests (1-second window)",
        xlabel="Time (seconds)", ylabel="Latency (seconds)",
        color="purple", logy=True, grid=True,
        xticks=range(0, int(df.index.max()) + 1, 60),
        yticks=[0.001, 0.010, 0.100, 1.0, 10.0, 50.0])

In [ ]:
fig = plt.figure(figsize=(24, 24))
for (i, (request_type, _)) in enumerate(requests.groupby(["type"])):
    df = requests[(requests["status"] == "successful") & (requests["type"] == request_type)]
    df = df.groupby(["window"])["latency"].agg(PIT_AGGREGATE_FUNC)
    df = df.reindex(range(0, int(requests["time"].max()) + 1), fill_value=0)
    ax = fig.add_subplot(4, 4, i + 1)
    ax.axvline(x=workload_conf["duration"]["ramp_up"],
               ls="--", color="green")
    ax.axvline(x=workload_conf["duration"]["total"] - workload_conf["duration"]["ramp_down"],
               ls="--", color="green")
    ax.grid(alpha=0.75)
    df.plot(ax=ax, kind="bar", title=request_type,
            xlabel="", ylabel="",
            color="purple", logy=True, grid=True,
            xticks=range(0, int(requests["time"].max()) + 1, 60),
            yticks=[0.001, 0.010, 0.100, 1.0, 10.0, 50.0])

### PIT Latency of Successful Requests (fine-grained window)

In [ ]:
# [CONFIG] Request type or '*' for all
REQUEST_TYPE = '*'

df = requests[requests["type"] == REQUEST_TYPE] if REQUEST_TYPE != '*' else requests
df = df[requests["status"] == "successful"]
df = df.groupby(["fg_window"])["latency"].agg(PIT_AGGREGATE_FUNC)
df = df.reindex(range(0, int(df.index.max()) + 1), fill_value=0)
fig = plt.figure(figsize=(108, 48))
ax = fig.gca()
ax.grid(alpha=0.75)
ax.axvline(x=(workload_conf["duration"]["ramp_up"] * 1000) // PIT_FG_WINDOW_IN_MS,
           ls="--", linewidth=5, color="green")
ax.axvline(x=((workload_conf["duration"]["total"] - workload_conf["duration"]["ramp_down"])  * 1000) // PIT_FG_WINDOW_IN_MS,
           ls="--", linewidth=5, color="green")
ax.set_xticklabels([str(x * 60) for x in range(0, int(df.index.max()) // (60 * (1000 // PIT_FG_WINDOW_IN_MS)) + 1)])
df.plot(ax=ax, kind="bar",
        title="PIT Latency of Successful Requests (%s-millisecond window)" % PIT_FG_WINDOW_IN_MS,
        xlabel="Time (seconds)", ylabel="Latency (seconds)",
        color="purple", logy=True, grid=True,
        xticks=range(0, int(df.index.max()) + 1, 60 * (1000 // PIT_FG_WINDOW_IN_MS)),
        yticks=[0.001, 0.010, 0.100, 1.0, 10.0, 50.0])

## System Event Monitoring Logs

In [ ]:
# Parse tcplife logs
conn_to = []
for node_hostname in os.listdir(os.path.join(EXPERIMENT_DIR, "logs")):
    if os.path.isfile(os.path.join(EXPERIMENT_DIR, "logs", node_hostname, "tcplife-bpfcc.tar.gz")):
        tarball = tarfile.open(os.path.join(EXPERIMENT_DIR, "logs", node_hostname, "tcplife-bpfcc.tar.gz"))
        with tarball.extractfile("./tcplife-bpfcc.log") as tcplife_log_file:
            reader = csv.DictReader(codecs.getreader("utf-8")(tcplife_log_file))
            for row in reader:
                if row["COMM"] in ("uwsgi", "nginx", "postgres", "account_server", "follow_server", "like_server", "post_server", "uniquepair_serv"):
                    conn_to.append({
                        "time": float(row["TIME(s)"]),
                        "comm": row["COMM"],
                        "addr": row["RADDR"],
                        "port": row["RPORT"],
                        "duration": float(row["MS"])
                    })

In [ ]:
# Build data frame
conn_to = pd.DataFrame(conn_to)
conn_to["fg_window"] = conn_to.apply(lambda r: range(int(r["time"] * 1000) // TCPLIFE_FG_WINDOW_IN_MS, int((r["time"] + r["duration"] / 1000) * 1000) // TCPLIFE_FG_WINDOW_IN_MS + 1), axis=1)
conn_to = conn_to.explode("fg_window")

### API Gateway (fine-grained Window)

In [ ]:
# [TODO] Get ports from configuration file

df = conn_to
df = df[df["port"] == "8080"]
df = df.groupby(["fg_window"])["fg_window"].count()
df = df.reindex(range(0, int(df.index.max()) + 1), fill_value=0)
fig = plt.figure(figsize=(108, 48))
ax = fig.gca()
ax.grid(alpha=0.75)
ax.axvline(x=(workload_conf["duration"]["ramp_up"] * 1000) // TCPLIFE_FG_WINDOW_IN_MS,
           ls="--", linewidth=5, color="green")
ax.axvline(x=((workload_conf["duration"]["total"] - workload_conf["duration"]["ramp_down"])  * 1000) // TCPLIFE_FG_WINDOW_IN_MS,
           ls="--", linewidth=5, color="green")
ax.set_xticklabels([str(x * 60) for x in range(0, int(df.index.max()) // (60 * (1000 // TCPLIFE_FG_WINDOW_IN_MS)) + 1)])
df.plot(ax=ax, kind="bar",
        title="API Gateway - Number of requests being processed (%s-millisecond window)" % TCPLIFE_FG_WINDOW_IN_MS,
        xlabel="Time", ylabel="Requests (count)",
        color="black", grid=True,
        xticks=range(0, int(df.index.max()) + 1, 60 * (1000 // TCPLIFE_FG_WINDOW_IN_MS)))

## Experiment Configuration

### System Configuration File

In [ ]:
print(yaml.dump(system_conf))

### Workload Configuration File

In [ ]:
print(yaml.dump(workload_conf))